Using the connection string to Azure Application Insights directly:

Go to Azure AI Foundry portal and select Tracing.

Select Manage data source. In this screen you can see the instance that is associated with the project.

Copy the value at Connection string and set it to the following variable:

```bash
export CONNECTION_STRING="Instrumentation..."
```

### Enable tracing for Langchain
You can enable tracing for Langchain that follows OpenTelemetry standards as per [opentelemetry-instrumentation-langchain](https://pypi.org/project/opentelemetry-instrumentation-langchain/). To enable tracing for Langchain, install the package `opentelemetry-instrumentation-langchain` using your package manager, like pip:

```bash
pip install opentelemetry-instrumentation-langchain
```

In [ ]:
import os
from langchain_azure_ai.callbacks.tracers.inference_tracing import AzureAIInferenceTracer
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_azure_ai.callbacks.tracers import AzureAIInferenceTracer

# os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true" # False by default

# Full langchain traces
# read more here https://pypi.org/project/opentelemetry-instrumentation-langchain/

# Get the resource name from environment variables

model = AzureChatOpenAI(model="gpt-4o", azure_deployment="gpt-4o", temperature=0.0, api_version="2024-10-21")

system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

parser = StrOutputParser()

chain = prompt_template | model | parser

# get the connection string from environment variables
application_insights_connection_string = os.environ["CONNECTION_STRING"]
# create the tracer with the connection string and enable content recording
tracer = AzureAIInferenceTracer(connection_string=application_insights_connection_string,
                                enable_content_recording=True,
)

# add the tracer to the LLM
chain = chain.with_config({"callbacks": [tracer]})
chain.invoke({"language": "spanish", "text": "hi"})


Using LangChain OpenAI the following can be seen:



![Langchain Traces in Azure AI Foundry portal](../img/ai_foundry_traces.png)

Let's go back to the [main file](../README.md#Production)